# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
credit = pd.read_csv('/datasets/data.csv')


In [2]:
credit.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


* *переименуем некоторые столбцы, для лучшего восприятия*
столбцы days_employed, dob_years, income_type, total_income, family_status	 в

 day, years,	type, salary, family

In [3]:
credit = credit.rename(columns={'days_employed':'day', 'income_type':'type', 'total_income':'salary', 'dob_years': 'years', 'family_status': 'family'})
credit.head(15)

,children,day,years,education,education_id,family,family_status_id,gender,type,debt,salary,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


* *по выводу первых строк видно что есть пропуски, не все строки приведены к одному регистру, в колонке day есть и положительные и отрицательные числа*

In [4]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
day                 19351 non-null float64
years               21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family              21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
debt                21525 non-null int64
salary              19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



* *Наблюдение:в столбцах day и salary  указаны не все строки*

In [5]:
credit.describe()

,children,day,years,education_id,family_status_id,debt,salary
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


* *Наблюдение: в колонке дети: min отрицательное число, как и max равное 20, что тоже маловероятно, скорее всего опечатка при занесение данныхю. В колонке day: и отрицательные числа и положительные, можно взять их по модулю. Также в этом столбце max равен практически 1100 лет, это ошибочные данные и они завышают как среднее, так и медиану столбца. В колонке years минимальное значение равно 0, что не может быть правдой В колонке salary: непонятное значение, можно округлить вещественные числа до целочисленных, такое же можно сделать и с колонкой day*

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
credit['day'] = abs(credit['day'])
credit['salary'] = abs(credit['salary'])
credit['children'] = abs(credit['children'])
credit['day'] = credit['day'].fillna(credit['day'].median())
credit['salary'] = credit['salary'].fillna(credit['salary'].median())
credit.info()
credit.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
day                 21525 non-null float64
years               21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family              21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
debt                21525 non-null int64
salary              21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,day,years,education,education_id,family,family_status_id,gender,type,debt,salary,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


* *Для заполнения пропусков, взял значения столбцов day и salary, по модулю и заменил пропуски на медианные значения*

### Замена типа данных

In [7]:
credit['education'] = credit['education'].str.lower()
credit['day'] = credit['day'].astype('int')
credit['salary'] = credit['salary'].astype('int')
credit.head(15)

,children,day,years,education,education_id,family,family_status_id,gender,type,debt,salary,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


* *перевел столбец education в нижний регистр, столбцы  day и salary в целоисчислительный тип данных*

### Обработка дубликатов

In [8]:
credit.duplicated().sum()

71

In [9]:
credit = credit.drop_duplicates()
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
day                 21454 non-null int64
years               21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family              21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
type                21454 non-null object
debt                21454 non-null int64
salary              21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [10]:
credit.duplicated().sum()

0

* *удалил полные дубликаты*

In [11]:
credit['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [12]:
credit['years'].value_counts()

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: years, dtype: int64

In [13]:
credit['years']=credit['years'].replace(0,credit['years'].median())

In [14]:
credit['years'].unique()  

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

### Лемматизация

In [15]:


def lemm(i):
    lemmas = m.lemmatize(i)                     
    return lemmas

credit['purpose_lemmas'] = credit['purpose'].apply(lemm)
credit.head(15)

,children,day,years,education,education_id,family,family_status_id,gender,type,debt,salary,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


* *применил лемматизацию к колонке purpose, создал новую колонку со значениями*

### Категоризация данных

In [16]:
credit['purpose_lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [17]:

def target(i):
    if 'свадьба' in i:
        return 'свадьба'
    elif 'автомобиль' in i or 'транспорт' in i:
        return 'авто'
    elif 'недвижимость' in i or 'жилье' in i:
        return 'недвижимость'
    elif 'образование' or 'профессия' in i:
        return 'образование'
    else:
        return 5
    
credit['purpose_id'] = credit['purpose_lemmas'].apply(target)
credit['purpose_id'].value_counts()



def money(salary):
    if salary <= 50000:
        return 'низкий'
    if 50000 < salary <=100000:
        return ' ниже среднего'
    if 100000 < salary <=160000:
        return 'средний'
    if 160000 < salary <=200000:
        return 'выше среднего'
    if salary > 200000:
        return 'высокий'
credit['salary_rang'] = credit['salary'].apply(money)    




credit.head(15)

,children,day,years,education,education_id,family,family_status_id,gender,type,debt,salary,purpose,purpose_lemmas,purpose_id,salary_rang
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",авто,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,ниже среднего
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний


* *Сделал категоризацию по цели кредита, а также по уровню дохода. Создал для них отдельные столбцы*

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
def conv_child(x):
    return "{0:.2%}".format(x)

debt_children = pd.DataFrame()

debt_children['count_children'] = credit.groupby('children')['debt'].count()
debt_children['sum_children'] = credit.groupby('children')['debt'].sum()
debt_children['result_children'] = debt_children['sum_children'] / debt_children['count_children'] 
debt_children['result_children'] = debt_children['result_children'].apply(conv_child)
debt_children.sort_values('result_children', ascending=False)

,count_children,sum_children,result_children
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4855,445,9.17%
3,330,27,8.18%
0,14091,1063,7.54%
20,76,8,10.53%
5,9,0,0.00%


* *Создал новый датафрейм для подсчета зависимости между наличием детей и возврата кредита в срок. Сначала сгруппировал столбцы задолженность и дети, посчитал общее количество, потом сумму должников по каждой категории. И затем нашел соотношение числа людей с просрочкой кредита к общему числу. Хоть и суммарное количество в категории людей без детей самое большое, но по процентному соотношению они самые надежные, в плане возврата кредита. Наверное, это связано с тем что у них дополнительных расходов на содержание ребенка, на втором месте, идут семьи с 3 детьми, они реже берут кредит, но и более ответственнены, чем семьи с 1-2 детьми, их процент просрочки примерно одинаков*


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
def conv_fam(x):
    return "{0:.2%}".format(x)

debt_family = pd.DataFrame()
debt_family['count_family'] = credit.groupby('family')['debt'].count()
debt_family['sum_family'] = credit.groupby('family')['debt'].sum()
debt_family['result_family'] = debt_family['sum_family'] / debt_family['count_family']
debt_family['result_family'] = debt_family['result_family'].apply(conv_fam)
debt_family.sort_values('result_family', ascending =False)

,count_family,sum_family,result_family
family,,,
Не женат / не замужем,2810,274,9.75%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%


* *Самые проблемные в плане возврата кредита, это холостые люди, либо состоящие в гражданском браке. Самые ответственные это потерявшие супруга/ги, что легко обьяснимо, они реже берут кредит, но если и берут, то здраво оценивают возможность возврата кредита.*

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
def conv_sal(x):
    return "{0:.2%}".format(x)
debt_salary = pd.DataFrame()
debt_salary['count_salary'] = credit.groupby('salary_rang')['debt'].count()
debt_salary['sum_salary'] = credit.groupby('salary_rang')['debt'].sum()
debt_salary['conversion_salary'] = debt_salary['sum_salary'] / debt_salary['count_salary']
debt_salary['conversion_salary'] = debt_salary['conversion_salary'].apply(conv_sal)
debt_salary.sort_values('conversion_salary', ascending = False)


,count_salary,sum_salary,conversion_salary
salary_rang,,,
выше среднего,3152,280,8.88%
средний,8773,749,8.54%
ниже среднего,4091,331,8.09%
высокий,5066,358,7.07%
низкий,372,23,6.18%


* *По данному сравнению можно сделать вывод, что люди с низким и ниже среднего доходом, более ответственны к выплате, чем люди со средним, либо чуть выше среднего дохода. С высоким доходом, категория самая надежная в плане выдачи кредита*

**Вывод**

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
def conv_sal(x):
    return "{0:.2%}".format(x)
debt_purpose = pd.DataFrame()
debt_purpose['count_purpose'] = credit.groupby('purpose_id')['debt'].count()
debt_purpose['sum_purpose'] = credit.groupby('purpose_id')['debt'].sum()
debt_purpose['conversion_purpose'] = debt_purpose['sum_purpose'] / debt_purpose['count_purpose']
debt_purpose['conversion_purpose'] = debt_purpose['conversion_purpose'].apply(conv_sal)
debt_purpose.sort_values('conversion_purpose', ascending = False)


,count_purpose,sum_purpose,conversion_purpose
purpose_id,,,
авто,4306,403,9.36%
образование,4013,370,9.22%
свадьба,2324,186,8.00%
недвижимость,10811,782,7.23%


* *По полученным данным, следует, что самой проблемной целью для получения кредита, является покупка авто, самое надежное это недвижимость, что также легко обьясняется, покупка недвижимости более ответственный шаг для человека, либо семьи, также покупка авто не учитывает, будущих расходов на содержание автомобиля, что упускают многие будущие владельцы*

## Шаг 4. Общий вывод

Подведем итог, из данной нам таблицы с клиентами, проведя категоризацию, выделили основные черты идеального клиента и его цели получения кредита. В категории наличия детей, лучшим, в плане возврата, является категория бездетных и семьи с 3 детьми, самыми проблемными семьи с 1-2 детьми. Выделив категорию семейного положения, мы узнали что самыми надежными являются вдовы/цы либо в разводе, а самые сложные это холостые, либо находящиеся в гражданском браке. По уровню дохода, люди, имеющие доход нижесреднего, наиболее надежные, а которые имеют средний и чуть вышесреднего доход,допускают больше всего просрочек по кредиту. Также проведя, лемматизацию, узнали 4 основных цели получения кредита: самой надежной целью является покупка недвижимости, на втором месте- свадьба, затем - образование, и на последнем месте покупка автомобиля. Мои рекомендации, что можно поправить, это в первую очередь, корректное заполнение колонок со стажем и доходом, почему то часть из них отрицательны. Во вторых, заполнять возраст, не оставляя нули в категории, что усложняет делать правильную категоризацию.